In [ ]:
#curl -X GET "https://api.domain.com.au/v1/listings/6311594" -H "accept: application/json" -H "X-Api-Key: key_6ec15e2b8eb8204065bb24faa31f4921"


In [2]:
# import requests
# url  = 'https://api.domain.com.au/v1/listings/6311594'
# headers = {'accept': 'application/json','X-Api-key':'key_6ec15e2b8eb8204065bb24faa31f4921'}
# response = requests.get(url,headers = headers)
# data = response.json()
# data

{'addressParts': {'displayAddress': '6/65 Albert Crescent, Burwood NSW 2134',
  'displayType': 'fullAddress',
  'stateAbbreviation': 'nsw',
  'postcode': '2134',
  'suburb': 'Burwood',
  'suburbId': 7832,
  'unitNumber': '6',
  'streetNumber': '65',
  'street': 'Albert Crescent'},
 'advertiserIdentifiers': {'advertiserType': 'agency',
  'advertiserId': 4299,
  'contactIds': [1298529],
  'agentIds': ['A12957']},
 'bathrooms': 1,
 'bedrooms': 1,
 'carspaces': 0,
 'channel': 'residential',
 'dateAvailable': '2016-11-22T00:00:00+11:00',
 'dateListed': '2009-09-11T07:08:02Z',
 'dateUpdated': '2018-05-20T23:36:14.17Z',
 'description': '1 Bedroom unit, combined lounge and dining, neat kitchen and bathroom, share laundry, only minutes walk to station and shops.',
 'energyEfficiencyRating': 0,
 'geoLocation': {'latitude': -33.8777593, 'longitude': 151.1082707},
 'headline': 'Inspect Saturday 12/11/16 - 10:00am - 10:15am',
 'id': 6311594,
 'inspectionDetails': {'inspections': [],
  'pastInspecti

In [5]:
# import requests
# url  = 'https://api.domain.com.au/v1/addressLocators?searchLevel=Address&state=VIC'
# headers = {'accept': 'application/json','X-Api-key':'key_6ec15e2b8eb8204065bb24faa31f4921'}
# response = requests.get(url,headers = headers)
# data = response.json()
# data

{'type': 'https://developer.domain.com.au/docs/latest/conventions/access',
 'title': 'Not Authorized',
 'detail': 'Operation not permitted on project'}

In [553]:
import pandas as pd
post = []
postcode = pd.read_excel("../data/suburb.xltx")
surburb = postcode['Unnamed: 1']
surburb = surburb.dropna()
surburb = surburb.drop_duplicates().reset_index().drop('index',axis = 1).rename({'Unnamed: 1':'surburb'}, axis = 1)

In [ ]:
result_list = []
for item in surburb['surburb']:
    suburbs = item.split("-")
    for suburb in suburbs:
        suburb_replaced = suburb.replace(" ", "%20")
        print("request suburb id for: ", suburb_replaced)
        url = f"https://api.domain.com.au/v1/addressLocators?searchLevel=Suburb&suburb={suburb_replaced}&state=VIC"
        headers = {"accept": "application/json", "X-Api-Key": "key_a68db2bad8d7006e8aa1a5326d638dd8"}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print("request failed", response.reason)
            continue
        data = response.json()
        suburb_id_list = [suburb_info["ids"][0]["id"] for suburb_info in data]
        for suburb_id in suburb_id_list:
            result_list.append([suburb, suburb_id])
df2.to_csv("../data/suburb_id.csv", index=False)


In [ ]:
df2 = pd.read_csv("../data/suburb_id.csv")
headers = {"accept": "application/json", "X-Api-Key": "key_a68db2bad8d7006e8aa1a5326d638dd8"}
agent_list = []
for index, item in df2[:128].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list1.csv')

In [150]:
headers = {"accept": "application/json", "X-Api-Key": "key_a68db2bad8d7006e8aa1a5326d638dd8"}
agent_list = []
for index, item in df2[128:].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list2.csv')

In [702]:
def to_int(text):
    if len(text.split(' ')) == 1:
        return int(text)
    else:
        return 0

In [727]:
df1 = pd.read_csv('../data/agent_list.csv',on_bad_lines='skip')
df1 = df1[~(df1['numberForRent'].isnull())]
df1['numberForRent'] = df1['numberForRent'].apply(to_int)
df2 = pd.read_csv('../data/agent_list2.csv',on_bad_lines='skip')
agent_info = pd.concat([df1, df2]).reset_index().drop(['index',	'Unnamed: 0'], axis = 1)
agent_info = agent_info[agent_info['numberForRent'] > 0]
agent_info_id = agent_info['id']
agent_info_id = agent_info_id.drop_duplicates()
agent_info_id.to_csv('../data/agent_info_id.csv')

In [728]:
agent_info_id = pd.read_csv('../data/agent_info_id.csv')
agent_info_id = agent_info_id.drop('Unnamed: 0', axis = 1)

In [ ]:
import requests
house_list = []
headers = {'accept': 'application/json','X-Api-key':'key_2064b0fe29a00f6a734ff057907cb717'}
for i in agent_info_id.id[850:]:
    print(i)
    url = f'https://api.domain.com.au/v1/agencies/{i}/listings?listingStatusFilter=liveAndArchived&pageNumber=2&pageSize=10000'
    response = requests.get(url,headers = headers)
    data = response.json()
    if len(data) == 0:
        continue
    if isinstance(data, dict):
        house_list.extend([data])
    else:
        house_list.extend(data)

In [324]:
df = pd.DataFrame(house_list)
df.to_csv('../data/raw/house_list14.csv')

In [470]:
import pandas as pd
house_data = pd.read_csv('../data/raw/house_list.csv')
house_data2 = pd.read_csv('../data/raw/house_list2.csv')
house_data3= pd.read_csv('../data/raw/house_list3.csv')
house_data4= pd.read_csv('../data/raw/house_list4.csv')
house_data5= pd.read_csv('../data/raw/house_list5.csv')
house_data6= pd.read_csv('../data/raw/house_list6.csv')
house_data7= pd.read_csv('../data/raw/house_list7.csv')
house_data8= pd.read_csv('../data/raw/house_list8.csv')
house_data9= pd.read_csv('../data/raw/house_list9.csv')
house = pd.concat([house_data, house_data2, house_data3, house_data4, house_data5, house_data6, house_data7, house_data8, house_data9])

/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (14,19,23,27,28,30,32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (18,27,28,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,18,31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
house_data10 = pd.read_csv('../data/raw/house_list10.csv')
house_data11 = pd.read_csv('../data/raw/house_list11.csv')
house_data12 = pd.read_csv('../data/raw/house_list12.csv')
house_data13 = pd.read_csv('../data/raw/house_list13.csv')
house_data13 = house_data13[:5730]
house_data14= pd.read_csv('../data/raw/house_list14.csv')
house_2 = pd.concat([house_data10,house_data11, house_data12,house_data13,house_data14])

In [471]:
house_df = house[['propertyTypes','objective','addressParts','advertiserIdentifiers','bathrooms','bedrooms', 'carspaces','dateUpdated', 'dateListed','geoLocation','priceDetails','landAreaSqm','dateAvailable']]

In [ ]:
house_df_2 = house_2[['propertyTypes','objective','addressParts','advertiserIdentifiers','bathrooms','bedrooms', 'carspaces','dateUpdated', 'dateListed','geoLocation','priceDetails','landAreaSqm','dateAvailable']]
house_df_2

In [472]:
house_df = house_df[house_df['objective'] == 'rent']

In [ ]:
house_df_2 = house_df_2[house_df_2['objective'] == 'rent']
house_df_2

In [473]:
house_df = house_df.reset_index().drop('index', axis = 1)
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable
0,['industrialWarehouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,0.0,2005-02-24T13:02:38.42Z,2004-01-23T02:22:00Z,"{'latitude': -37.842632, 'longitude': 144.940494}","{'price': 32000.0, 'canDisplayPrice': True, 'd...",NaN,NaN
1,['industrialWarehouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,1.0,2005-09-19T14:31:05.103Z,2004-03-16T06:20:00Z,"{'latitude': -37.831794, 'longitude': 144.951065}","{'price': 50000.0, 'canDisplayPrice': True, 'd...",NaN,NaN
2,['industrialWarehouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,0.0,2005-03-05T05:26:45.063Z,2004-04-20T07:10:00Z,"{'latitude': -37.836438, 'longitude': 144.969564}","{'price': 60000.0, 'canDisplayPrice': True, 'd...",NaN,NaN
3,['industrialWarehouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,0.0,2005-07-29T12:56:53.75Z,2004-06-04T06:42:00Z,"{'latitude': -37.832703, 'longitude': 144.948314}","{'price': 19500.0, 'canDisplayPrice': True, 'd...",NaN,NaN
4,['industrialWarehouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,6.0,2005-11-09T13:26:33.187Z,2004-07-13T10:11:00Z,"{'latitude': -37.840653, 'longitude': 144.940246}","{'price': 2684.0, 'canDisplayPrice': True, 'di...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
333013,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,"{'latitude': -36.7365174, 'longitude': 147.159...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-01
333014,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,"{'latitude': -36.73394589999999, 'longitude': ...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-15
333015,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,"{'latitude': -36.6861761, 'longitude': 147.137...","{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,2022-06-01
333016,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,"{'latitude': -36.7329424, 'longitude': 147.152...","{'canDisplayPrice': False, 'displayPrice': '$5...",534.0,2022-07-05


In [421]:
house_df_2 = house_df_2.reset_index().drop('index', axis = 1)
house_df_2

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable
0,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,0.0,2005-07-20T10:01:03.573Z,2005-07-11T10:41:17Z,"{'latitude': -37.83775, 'longitude': 144.941043}","{'price': 600.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
1,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,3.0,1.0,2005-08-18T06:05:13.013Z,2005-07-13T09:41:48Z,"{'latitude': -37.834815, 'longitude': 144.960958}","{'price': 400.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
2,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,1.0,2005-07-19T05:28:18Z,2005-07-14T14:01:41Z,"{'latitude': -37.842189, 'longitude': 144.95752}","{'price': 550.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
3,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,2.0,2005-07-22T11:35:45.783Z,2005-07-14T14:01:53Z,"{'latitude': -37.851494, 'longitude': 144.956552}","{'price': 330.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
4,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,2.0,2007-08-28T00:49:03Z,2005-07-15T05:19:40Z,"{'latitude': -37.839805, 'longitude': 144.927453}","{'price': 875.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152899,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2018-08-30T23:14:14.3Z,2018-08-23T01:15:12Z,"{'latitude': -36.0428871, 'longitude': 146.935...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152900,['townhouse'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,2.0,2018-09-26T03:12:36.243Z,2018-08-24T08:16:04Z,"{'latitude': -36.0384167, 'longitude': 146.938...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152901,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2018-09-06T06:13:17.627Z,2018-08-29T07:14:29Z,"{'latitude': -36.0349289, 'longitude': 146.930...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152902,['townhouse'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-22T01:12:51.707Z,2020-05-12T05:23:21Z,"{'latitude': -36.0454958, 'longitude': 146.913...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN


In [474]:
house_df.to_csv('../data/raw/house_df.csv')

In [422]:
house_df_2.to_csv('../data/raw/house_df_2.csv')

In [475]:
import re
suburb_df = []
for i in range(len(house_df['addressParts'])):
    s = re.findall(r'\'suburb\': +\'+[A-Za-z\' \']+',house_df['addressParts'][i])
    if(len(s) == 0):
        suburb_df.append(' ')
    else:
        s = s[0].split(':')[1][2:-1].lower()
        suburb_df.append(s)

In [424]:
import re
suburb_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    s = re.findall(r'\'suburb\': +\'+[A-Za-z\' \']+',house_df_2['addressParts'][i])
    if(len(s) == 0):
        suburb_df_2.append(' ')
    else:
        s = s[0].split(':')[1][2:-1].lower()
        suburb_df_2.append(s)

In [476]:
postcode_df = []
for i in range(len(house_df['addressParts'])):
    postcode = re.findall(r'postcode\': \'[0-9]+',house_df['addressParts'][i])
    if(len(postcode) == 0):
        postcode_df.append(0)
    else:
        postcode = postcode[0].split(':')[1][2:]
        postcode_df.append(int(postcode))

In [425]:
postcode_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    postcode = re.findall(r'postcode\': \'[0-9]+',house_df_2['addressParts'][i])
    if(len(postcode) == 0):
        postcode_df_2.append(0)
    else:
        postcode = postcode[0].split(':')[1][2:]
        postcode_df_2.append(int(postcode))

In [478]:
suburb_df = pd.DataFrame(suburb_df)
postcode_df = pd.DataFrame(postcode_df)
house_df['postcode'] = postcode_df
house_df['surburb'] = suburb_df

In [ ]:
suburb_df_2 = pd.DataFrame(suburb_df_2)
postcode_df_2 = pd.DataFrame(postcode_df_2)
house_df_2['postcode'] = postcode_df_2
house_df_2['surburb'] = suburb_df_2

In [479]:
year_df = []
month_df = []
for i in range(len(house_df['addressParts'])):
    date = house_df['dateUpdated'][i]
    if(len(date) == 0):
        year_df.append(0)
        month_df.append(0)
    else:
        year = date[:4]
        month = date[5:7]
        year_df.append(int(year))
        month_df.append(int(month))


In [429]:
year_df_2 = []
month_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    date = house_df_2['dateUpdated'][i]
    if(len(date) == 0):
        year_df_2.append(0)
        month_df_2.append(0)
    else:
        year = date[:4]
        month = date[5:7]
        year_df_2.append(int(year))
        month_df_2.append(int(month))

In [480]:
year_df = pd.DataFrame(year_df)
month_df = pd.DataFrame(month_df)
house_df['year'] = year_df
house_df['month'] = month_df

In [430]:
year_df_2 = pd.DataFrame(year_df_2)
month_df_2 = pd.DataFrame(month_df_2)
house_df_2['year'] = year_df_2
house_df_2['month'] = month_df_2

In [481]:
house_df = house_df[house_df['year']>2018]
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month
4256,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{'latitude': -37.8341628, 'longitude': 144.960...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3205,south melbourne,2019,1
5429,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,"{'latitude': -37.8443239, 'longitude': 144.943...","{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3207,port melbourne,2019,1
5437,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,"{'latitude': -37.8430561, 'longitude': 144.94026}","{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1
5439,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,"{'latitude': -37.846166, 'longitude': 144.957026}","{'canDisplayPrice': False, 'displayPrice': '$9...",NaN,NaN,3206,albert park,2019,1
5447,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,"{'latitude': -37.8408338, 'longitude': 144.939...","{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
333013,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,"{'latitude': -36.7365174, 'longitude': 147.159...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-01,3698,tawonga south,2022,3
333014,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,"{'latitude': -36.73394589999999, 'longitude': ...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-15,3698,tawonga south,2022,5
333015,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,"{'latitude': -36.6861761, 'longitude': 147.137...","{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,2022-06-01,3697,tawonga,2022,5
333016,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,"{'latitude': -36.7329424, 'longitude': 147.152...","{'canDisplayPrice': False, 'displayPrice': '$5...",534.0,2022-07-05,3698,tawonga south,2022,6


In [431]:
house_df_2 = house_df_2[house_df_2['year']>2018]
house_df_2 = house_df_2.reset_index()
house_df_2['year'].value_counts()

2021    9894
2022    9267
2019    8940
2020    7892
Name: year, dtype: int64

In [482]:
house_df = house_df.reset_index()
house_df['year'].value_counts()

2021    22877
2020    20744
2019    19223
2022    17454
Name: year, dtype: int64

In [483]:
def get_bond(text):
    text_dict = eval(text)
    return text_dict.get('bond')
def get_price(text):
    text_dict = eval(text)
    if text_dict.get('price'):
        return text_dict.get('price')
    else:
        return text_dict.get('displayPrice')

def to_lower(text):
    if isinstance(text,float):
        return text
    text = text.lower()
    return text

In [484]:
house_df['price'] = house_df['priceDetails'].apply(get_price)
house_df['bond'] = house_df['priceDetails'].apply(get_bond)
house_df['price'] = house_df['price'].apply(to_lower)

In [432]:
house_df_2['price'] = house_df_2['priceDetails'].apply(get_price)
house_df_2['bond'] = house_df_2['priceDetails'].apply(get_bond)
house_df_2['price'] = house_df_2['price'].apply(to_lower)

In [357]:
def get_weekly_price(text):
    if isinstance(text,float):
        return text
    money = re.findall(r'\$\d*[,.]?\d+[,.]?\d+',text)
    if len(money) == 0:
        return None
    elif len(money) == 1:
        money = money[0].replace(',','')[1:]
        if money.count('.') == 2:
            return None
        else:
            return float(money)
    else:
        if('per week' in text):
            num_text = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ per week',text)
            if (len(num_text)== 1):
                return float(num_text[0].split(' ')[0][1:].replace(',',''))
            else:
                total = 0
                for i in num_text:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(m)
        elif('pw' in text):
            text1 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ pw',text)
            if(len(text1) == 1):
                return float(text1[0].split(' ')[0][1:].replace(',',''))
            elif(len(text1) > 1):
                total = 0
                for i in text1:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(text1)
            else:
                text2 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+pw',text)
                if(len(text2)==1):
                    if(text2[0][0] != '$'):
                        return float(text2[0][:-2].replace(',',''))
                    else:
                        return float(text2[0][1:-2].replace(',',''))
                elif(len(text2) > 1):
                    total = 0
                    for i in text2:
                        i = i.split(' ')[0][1:-2].replace(',','')
                        total += float(i)
                    return total/len(text2)
        elif('p/w' in text):
            text3 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ p/w',text)
            if(len(text3)==1):
                text3 = text3[0].split(' ')[0][1:].replace(',','')
                return float(text3)
            elif(len(text3)>1):
                total = 0
                for i in text3:
                    i = i.split(' ')[0][1:-2].replace(',','')
                    total += float(i)
                return total/len(text3)
            else:
                text4 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+p/w',text)
                if (len(text4)==1):
                    return float(text4[0][1:-3].replace(',',''))
                elif (len(text4) > 1):
                    total = 0
                    for i in text4:
                        i = i[1:-3].replace(',','')
                        total += float(i)
                    return total/len(text4)

                    

In [504]:
house_df['rent'] = house_df['price'].apply(get_weekly_price)
house_df.to_csv("../data/curated/house_df.csv")

In [503]:
house_df

,index,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent
0,4256,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0
1,5429,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0
2,5437,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0
3,5439,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,...,"{'canDisplayPrice': False, 'displayPrice': '$9...",NaN,NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0
4,5447,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80293,333013,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,...,"{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-01,3698,tawonga south,2022,3,$450 unfurnished or $500 furnished,NaN,NaN
80294,333014,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,...,"{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-15,3698,tawonga south,2022,5,$490 per week,NaN,490.0
80295,333015,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,2022-06-01,3697,tawonga,2022,5,$500 per week,NaN,500.0
80296,333016,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",534.0,2022-07-05,3698,tawonga south,2022,6,$540 per week,NaN,540.0


In [505]:
house_df_2['rent'] = house_df_2['price'].apply(get_weekly_price)
house_df_2.to_csv("../data/curated/house_df2.csv")

In [506]:
house_df

,index,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent
0,4256,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0
1,5429,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0
2,5437,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0
3,5439,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,...,"{'canDisplayPrice': False, 'displayPrice': '$9...",NaN,NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0
4,5447,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80293,333013,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",1.0,2.0,1.0,2022-03-21T01:21:52.75Z,2022-03-09T02:28:07Z,...,"{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-01,3698,tawonga south,2022,3,$450 unfurnished or $500 furnished,NaN,NaN
80294,333014,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,2.0,2022-05-02T03:23:46.94Z,2022-03-23T04:13:37Z,...,"{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-04-15,3698,tawonga south,2022,5,$490 per week,NaN,490.0
80295,333015,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,3.0,1.0,2022-05-13T05:27:18.527Z,2022-05-10T00:57:02Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,2022-06-01,3697,tawonga,2022,5,$500 per week,NaN,500.0
80296,333016,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 3...",2.0,4.0,0.0,2022-06-22T01:30:54.673Z,2022-05-31T06:16:44Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",534.0,2022-07-05,3698,tawonga south,2022,6,$540 per week,NaN,540.0


In [563]:
house_df_combined = pd.concat([house_df,house_df_2]).reset_index().drop(['level_0'],axis = 1)

In [564]:
house_df_combined 

,index,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent
0,4256,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0
1,5429,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0
2,5437,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0
3,5439,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,...,"{'canDisplayPrice': False, 'displayPrice': '$9...",NaN,NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0
4,5447,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116286,152888,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-06-03T06:12:52.787Z,2020-04-28T05:13:10Z,...,"{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN,2641,lavington,2020,6,$210 per week,NaN,210.0
116287,152889,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2019-08-08T03:32:08.4Z,2019-07-26T04:13:41Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3690,wodonga,2019,8,$190 per week,NaN,190.0
116288,152894,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,4.0,2.0,2019-09-10T00:13:03.153Z,2019-08-08T11:13:16Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,2640,west albury,2019,9,$315 per week,NaN,315.0
116289,152895,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,2.0,2019-10-23T07:15:16Z,2019-10-03T05:12:50Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,2641,lavington,2019,10,$300 per week,NaN,300.0


In [566]:
import ast
state = []
for i in range(len(house_df_combined)): 
    s = ast.literal_eval(house_df_combined['addressParts'][i])['stateAbbreviation']
    state.append(s)

In [567]:
house_df_combined['state'] = state

In [568]:
house_df_combined = house_df_combined[house_df_combined['state'] == 'vic'].drop('index', axis = 1)
house_df_combined

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,...,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent,state
0,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,"{'latitude': -37.8341628, 'longitude': 144.960...",...,NaN,NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0,vic
1,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,"{'latitude': -37.8443239, 'longitude': 144.943...",...,NaN,NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0,vic
2,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,"{'latitude': -37.8430561, 'longitude': 144.94026}",...,NaN,NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0,vic
3,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,"{'latitude': -37.846166, 'longitude': 144.957026}",...,NaN,NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0,vic
4,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,"{'latitude': -37.8408338, 'longitude': 144.939...",...,NaN,NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0,vic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116252,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2021-03-25T22:25:48.957Z,2021-03-12T06:10:56Z,"{'latitude': -36.1394497, 'longitude': 146.882...",...,NaN,2021-04-05,3690,wodonga,2021,3,$215 per week,NaN,215.0,vic
116253,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2021-03-31T00:18:22.6Z,2021-03-18T00:21:42Z,"{'latitude': -36.1787142, 'longitude': 146.940...",...,NaN,2021-04-22,3691,baranduda,2021,3,$465 per week,NaN,465.0,vic
116254,['townhouse'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2021-03-30T22:40:44.093Z,2021-03-23T06:13:04Z,"{'latitude': -36.1298786, 'longitude': 146.885...",...,NaN,2021-03-31,3690,wodonga,2021,3,$320 per week,NaN,320.0,vic
116255,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,2.0,2021-04-08T22:10:17.477Z,2021-03-26T00:07:22Z,"{'latitude': -36.1398519, 'longitude': 146.887...",...,448.0,2021-05-05,3690,wodonga,2021,4,$360 per week,NaN,360.0,vic


In [569]:
house_df_combined.to_csv('../data/house_combined.csv')

In [601]:
house_df_combined = pd.read_csv('../data/house_combined.csv')

In [615]:
house_df_combined['longitude'][100]

144.9608734

In [625]:
pd.isnull(house_df_combined.geoLocation[9393])

True

In [631]:
import ast
lat_list = []
long_list = []
for i in range(len(house_df_combined)):
    if pd.isnull(house_df_combined.geoLocation[i]):
        lat_list.append(None)
        long_list.append(None)
    else:
        latitude = ast.literal_eval(house_df_combined.geoLocation[i])['latitude']
        longitude = ast.literal_eval(house_df_combined.geoLocation[i])['longitude']
        lat_list.append(latitude)
        long_list.append(longitude)


In [ ]:
lat_list

In [633]:
house_df_combined['latitude'] = lat_list
house_df_combined['longitude'] = long_list

In [639]:
coords = ((144.961602,-37.813937),(house_df_combined['longitude'][20],house_df_combined['latitude'][20]))
client = openrouteservice.Client(key='5b3ce3597851110001cf62487d02abe8bc754613932637f38235c688') 
routes = client.directions(coords, profile='driving-car', optimize_waypoints=True)
distance = routes['routes'][0]['summary']['distance']
distance

4775.7

In [642]:
for i in 40:len(house_df_combined):
    print(i)

SyntaxError: invalid syntax (1048848531.py, line 1)

In [653]:
import openrouteservice
# melbourne CBD coordinate (144.961602,-37.813937)
distance_list_5 = []
for i in range(279,len(house_df_combined)):
    print(i)
    coords = ((144.961602,-37.813937),(house_df_combined['longitude'][i],house_df_combined['latitude'][i]))
    client = openrouteservice.Client(key='5b3ce3597851110001cf62487d02abe8bc754613932637f38235c688') 
    routes = client.directions(coords, profile='driving-car', optimize_waypoints=True)
    distance = routes['routes'][0]['summary']['distance']
    print(distance)
    distance_list_5.append(distance)



279


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/directions.py:202: UserWarning: Less than 4 coordinates, nothing to optimize!
  warnings.warn("Less than 4 coordinates, nothing to optimize!", UserWarning)


2692.4
280
5631.2
281
4278.3
282
5736.2
283
4470.9
284
5945.2
285
5143.0
286
4766.3
287
3698.2
288
3388.9
289
4532.1
290
11160.7
291
4999.0
292
3847.2
293
5736.9
294
7418.8
295
5034.6
296
13583.4
297
5088.0
298
3331.2
299


/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyonglin/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/heyon

Timeout: 

In [645]:
distance_list

[2990.7,
 5175.7,
 5034.4,
 4758.3,
 4845.9,
 5204.0,
 5856.0,
 5488.4,
 5153.0,
 3445.0,
 4076.1,
 3871.4,
 3690.7,
 4426.5,
 4160.3,
 4490.0,
 3520.2,
 5162.4,
 4495.5,
 5053.6,
 4775.7,
 3877.5,
 3811.9,
 4196.2,
 5375.1,
 3074.4,
 4924.7,
 8074.9]

In [636]:
len(distance_list)

20

In [599]:
routes['routes'][0]['summary']['distance']
routes['routes'][0]['summary']['duration']

356.0